Goal: apply a DL model on different dataset
- Preprocess with MinMaxScaler
- Preprocess with CustomerTransformer

# Imports, variables, functions

In [1]:
import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from scipy.fftpack import rfft
from sklearn.metrics import accuracy_score
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from plot_keras_history import plot_history

2022-12-02 11:14:00.678528: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 11:14:01.161939: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 11:14:01.238235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 11:14:01.238258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [3]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(signals, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [4]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Add a time column with the seconds
def add_time(df):
    list_time=[]
    for i in range(len(df)):
        list_time.append(i//sampling_rate)
    df["time"]=list_time
    return df

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort', 'time'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    add_time(data_patient)
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [5]:
## -- MODEL FUNCTIONS --

def flatten_window(window_df):
    if len(np.unique(window_df.iloc[:,-1])) == 1:
        target = window_df.iloc[0,-1]
    else:
        target = 1
    t_df = window_df.drop(columns = "is_seizure_target").transpose()
    flatten = pd.DataFrame(np.array(t_df).reshape(1,t_df.shape[0]*t_df.shape[1]))
    flatten["Target"] = target
    return flatten

def create_data_input(df):
    data = np.array([flatten_window(df.iloc[i:i+len_window+1]) for i in range(0,len(df)-len_window, overlap*sample_rate)])
    r=data.shape[0]
    c=data.shape[2]
    
    data = pd.DataFrame(data.reshape(r,c))
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    return X, y

def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

# Pipeline

In [6]:
def model_pipeline(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)

    # Create data input
    X, y =  create_data_input(df)
    
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
    
    # Balancing
    X_train, y_train = oversampling(X_train, y_train)
    
    # Implement model
    model = Sequential()
    model.add(layers.Dense(20, activation='relu', input_dim=X_train.shape[1]))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy', 
        metrics=[ts.keras.metrics.Recall(),"accuracy"])
    history=model.fit(X_train,y_train, batch_size=32, epochs=40)
    # Evaluation
    results_DL =model.evaluate(X_test, y_test)
    loss=results_DL[0]
    recall = results_DL[1]
    accuracy = results_DL[2]
    
    return loss, recall, accuracy

# Test DL model on different datasets

## EEGs to test

In [ ]:
## -- DO NOT RUN THIS CELL --

#patients_numbers = list(range(1, 80))

#patient_with_issue = [4, 29, 50] # Can't import ECG4, ECG29 and ECG50
#patient_without_seizure = [3, 6, 10, 12, 18, 24, 26, 27, 28, 30, 32, 35, 37, 42, 43, 45, 46, 48, 49, 53, 55, 56, 57, 58, 59, 60, 61, 64, 65, 70, 72, 74]

#for i in patient_with_issue:
 #   patients_numbers.remove(i)

#for i in patient_without_seizure:
 #   patients_numbers.remove(i)

## Test on Preprocess data with MinMaxScaler

In [7]:
# Test for 5 patients
loss_1,recall_1, accuracy_1 = model_pipeline("../raw_data/eeg1.edf",  MinMaxScaler(), 1)
loss_5,recall_5, accuracy_5 = model_pipeline("../raw_data/eeg5.edf",  MinMaxScaler(), 5)
loss_25,recall_25, accuracy_25 = model_pipeline("../raw_data/eeg25.edf",  MinMaxScaler(), 25)
#recall_44, f1_score_44 = model_pipeline("../raw_data/eeg44.edf",  MinMaxScaler(), 44)
#recall_71, f1_score_71 = model_pipeline("../raw_data/eeg71.edf",  MinMaxScaler(), 71)

2022-12-02 11:15:52.334594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-02 11:15:52.350211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 11:15:52.350895: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-02 11:15:52.351860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-02 11:15:52.352014: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Epoch 1/40
52/52 [==============================] - 3s 13ms/step - loss: 1.7896 - recall: 0.5749 - accuracy: 0.5469
Epoch 2/40
52/52 [==============================] - 1s 10ms/step - loss: 1.0059 - recall: 0.6407 - accuracy: 0.6516
Epoch 3/40
52/52 [==============================] - 0s 9ms/step - loss: 0.4968 - recall: 0.8076 - accuracy: 0.7783
Epoch 4/40
52/52 [==============================] - 1s 10ms/step - loss: 0.5206 - recall: 0.7868 - accuracy: 0.7643
Epoch 5/40
52/52 [==============================] - 1s 11ms/step - loss: 0.6512 - recall: 0.7844 - accuracy: 0.7808
Epoch 6/40
52/52 [==============================] - 1s 12ms/step - loss: 0.6360 - recall: 0.7674 - accuracy: 0.7570
Epoch 7/40
52/52 [==============================] - 1s 10ms/step - loss: 0.5089 - recall: 0.8149 - accuracy: 0.8021
Epoch 8/40
52/52 [==============================] - 0s 9ms/step - loss: 0.3223 - recall: 0.8879 - accuracy: 0.8928
Epoch 9/40
52/52 [==============================] - 0s 9ms/step - loss: 0.

Epoch 30/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0087 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 31/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0081 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 32/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0079 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 33/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0072 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 34/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0070 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 35/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0063 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 36/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0060 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 37/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0058 - recall_1: 1.0000 - accuracy: 1.0000
Epoch 38/40
33/33 [=============================

In [8]:
print(f"patient 1:{loss_1, recall_1, accuracy_1}")
print(f"patient 5:{loss_5, recall_5, accuracy_5}")
print(f"patient 25:{loss_25, recall_25, accuracy_25}")

patient 1:(1.3372727632522583, 0.2666666805744171, 0.6535714268684387)
patient 5:(1.1054766178131104, 0.9200000166893005, 0.7727272510528564)
patient 25:(0.6758911609649658, 0.0, 0.9216417670249939)


In [ ]:
## -- DO NOT RUN THIS CELL --

#recall_minmax = []
#f1_score_minmax = []

#for i in patients_numbers:
 #   recall_i, f1_score_i = model_pipeline(f"../raw_data/eeg{i}.edf",  MinMaxScaler(), i)
  #  recall_minmax.append(recall_i)
   # f1_score_minmax.append(f1_score_i)

## Test on Preprocess data with CustomerTransformer

In [9]:
# Test for 5 patients
loss_1_custom,recall_1_custom, accuracy_score_1_custom = model_pipeline("../raw_data/eeg1.edf",  CustomTranformer(), 1)
loss_5_custom,recall_5_custom, accuracy_score_5_custom = model_pipeline("../raw_data/eeg5.edf",  CustomTranformer(), 5)
loss_25_custom,recall_25_custom, accuracy_score_25_custom = model_pipeline("../raw_data/eeg25.edf",  CustomTranformer(), 25)
#recall_44_custom, f1_score_44_custom = model_pipeline("../raw_data/eeg44.edf",  CustomTranformer(), 44)
#recall_71_custom, f1_score_71_custom = model_pipeline("../raw_data/eeg71.edf",  CustomTranformer(), 71)

Epoch 1/40
53/53 [==============================] - 2s 14ms/step - loss: 26.7443 - recall_3: 0.5829 - accuracy: 0.5823
Epoch 2/40
53/53 [==============================] - 1s 12ms/step - loss: 7.7276 - recall_3: 0.7168 - accuracy: 0.7281
Epoch 3/40
53/53 [==============================] - 1s 10ms/step - loss: 5.6147 - recall_3: 0.7630 - accuracy: 0.7897
Epoch 4/40
53/53 [==============================] - 1s 10ms/step - loss: 2.3940 - recall_3: 0.8175 - accuracy: 0.8329
Epoch 5/40
53/53 [==============================] - 1s 13ms/step - loss: 1.4010 - recall_3: 0.8791 - accuracy: 0.8868
Epoch 6/40
53/53 [==============================] - 1s 13ms/step - loss: 0.7053 - recall_3: 0.9111 - accuracy: 0.9135
Epoch 7/40
53/53 [==============================] - 1s 12ms/step - loss: 0.7872 - recall_3: 0.9005 - accuracy: 0.9117
Epoch 8/40
53/53 [==============================] - 1s 10ms/step - loss: 0.4940 - recall_3: 0.9384 - accuracy: 0.9378
Epoch 9/40
53/53 [==============================] - 1s 

33/33 [==============================] - 0s 10ms/step - loss: 0.2121 - recall_4: 0.9904 - accuracy: 0.9817
Epoch 30/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0260 - recall_4: 0.9981 - accuracy: 0.9846
Epoch 31/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0195 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 32/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0192 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 33/40
33/33 [==============================] - 0s 12ms/step - loss: 0.0189 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 34/40
33/33 [==============================] - 0s 14ms/step - loss: 0.0187 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 35/40
33/33 [==============================] - 0s 13ms/step - loss: 0.0185 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 36/40
33/33 [==============================] - 0s 14ms/step - loss: 0.0183 - recall_4: 1.0000 - accuracy: 0.9855
Epoch 37/40
33/33 [==============================] - 0s 10ms

In [13]:
print(f"patient 1:{loss_1_custom, recall_1_custom, accuracy_score_1_custom}")
print(f"patient 5:{loss_5_custom, recall_5_custom, accuracy_score_5_custom}")
print(f"patient 25:{loss_25_custom, recall_25_custom, accuracy_score_25_custom}")

patient 1:(29.168012619018555, 0.3253012001514435, 0.6607142686843872)
patient 5:(183.2772979736328, 0.7938931584358215, 0.6948052048683167)
patient 25:(60.49898147583008, 0.0, 0.89552241563797)


In [ ]:
## -- DO NOT RUN THIS CELL --

#recall_custom = []
#f1_score_custom = []

#for i in patients_numbers:
 #   recall_i, f1_score_i = model_pipeline(f"../raw_data/eeg{i}.edf",  CustomTransformer(), i)
  #  recall_custom.append(recall_i)
   # f1_score_custom.append(f1_score_i)

## Test on Preprocess data with CustomTransformer and Fourrier filter

In [17]:
loss_1_fourrier,recall_1_fourrier, accuracy_score_1_fourrier = model_pipeline("../raw_data/eeg1.edf",  CustomTranformer(), 1, Fournier=True)
loss_5_fourrier,recall_5_fourrier, accuracy_score_5_fourrier = model_pipeline("../raw_data/eeg5.edf",  CustomTranformer(), 5, Fournier=True)
loss_25_fourrier,recall_25_fourrier, accuracy_score_25_fourrier = model_pipeline("../raw_data/eeg25.edf",  CustomTranformer(), 25, Fournier=True)
#recall_44_fournier, f1_score_44_fournier = model_pipeline("../raw_data/eeg44.edf",  CustomTranformer(), 44, Fournier=True)
#recall_71_fournier, f1_score_71_fournier = model_pipeline("../raw_data/eeg71.edf",  CustomTranformer(), 71, Fournier=True)

Epoch 1/40
52/52 [==============================] - 2s 11ms/step - loss: 3700.2881 - recall_9: 0.4603 - accuracy: 0.5487
Epoch 2/40
52/52 [==============================] - 1s 11ms/step - loss: 466.5170 - recall_9: 0.5901 - accuracy: 0.6148
Epoch 3/40
52/52 [==============================] - 1s 11ms/step - loss: 114.8029 - recall_9: 0.7308 - accuracy: 0.7007
Epoch 4/40
52/52 [==============================] - 0s 9ms/step - loss: 30.3343 - recall_9: 0.8077 - accuracy: 0.8071
Epoch 5/40
52/52 [==============================] - 0s 9ms/step - loss: 16.1901 - recall_9: 0.8474 - accuracy: 0.8510
Epoch 6/40
52/52 [==============================] - 1s 10ms/step - loss: 12.3588 - recall_9: 0.8702 - accuracy: 0.8654
Epoch 7/40
52/52 [==============================] - 0s 9ms/step - loss: 7.1830 - recall_9: 0.9026 - accuracy: 0.9032
Epoch 8/40
52/52 [==============================] - 1s 10ms/step - loss: 2.1371 - recall_9: 0.9579 - accuracy: 0.9555
Epoch 9/40
52/52 [==============================]

Epoch 28/40
33/33 [==============================] - 0s 13ms/step - loss: 0.6364 - recall_10: 0.7243 - accuracy: 0.6553
Epoch 29/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6322 - recall_10: 0.6136 - accuracy: 0.6709
Epoch 30/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6245 - recall_10: 0.6738 - accuracy: 0.6796
Epoch 31/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6199 - recall_10: 0.6544 - accuracy: 0.6699
Epoch 32/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6175 - recall_10: 0.6388 - accuracy: 0.6786
Epoch 33/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6149 - recall_10: 0.6447 - accuracy: 0.6786
Epoch 34/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6118 - recall_10: 0.6175 - accuracy: 0.6874
Epoch 35/40
33/33 [==============================] - 0s 12ms/step - loss: 0.6098 - recall_10: 0.6777 - accuracy: 0.6883
Epoch 36/40
33/33 [=====================

In [18]:
print(f"patient 1:{loss_1_fourrier, recall_1_fourrier, accuracy_score_1_fourrier}")
print(f"patient 5:{loss_5_fourrier, recall_5_fourrier, accuracy_score_5_fourrier}")
print(f"patient 25:{loss_25_fourrier, recall_25_fourrier, accuracy_score_25_fourrier}")

patient 1:(886.3180541992188, 0.3380281627178192, 0.699999988079071)
patient 5:(0.6782591938972473, 0.7259259223937988, 0.701298713684082)
patient 25:(0.6894007921218872, 0.0, 0.9552238583564758)


In [ ]:
## -- DO NOT RUN THIS CELL --

#recall_fournier = []
#f1_score_fournier = []

#for i in patients_numbers:
 #   recall_i, f1_score_i = model_pipeline(f"../raw_data/eeg{i}.edf",  CustomTransformer(), i, Fournier=True)
  #  recall_fournier.append(recall_i)
   # f1_score_fournier.append(f1_score_i)

# Results

In [ ]:
# Test for 5 patients
patients_numbers = [1, 5, 25, 44, 71]
recall_minmax = [recall_1, recall_5, recall_25, recall_44, recall_71]
recall_custom = [recall_1_custom, recall_5_custom, recall_25_custom, recall_44_custom, recall_71_custom]
recall_fournier = [recall_1_fournier, recall_5_fournier, recall_25_fournier, recall_44_fournier, recall_71_fournier]
f1_score_minmax = [f1_score_1, f1_score_5, f1_score_25, f1_score_44, f1_score_71]
f1_score_custom = [f1_score_1_custom, f1_score_5_custom, f1_score_25_custom, f1_score_44_custom, f1_score_71_custom]
f1_score_fournier = [f1_score_1_fournier, f1_score_5_fournier, f1_score_25_fournier, f1_score_44_fournier, f1_score_71_fournier]

results  = pd.DataFrame(recall_minmax, patients_numbers)
results.rename(columns={0:'recall_minmax'}, inplace=True)
results['recall_custom'] = recall_custom
results['recall_fournier'] = recall_fournier
results['f1_score_minmax'] = f1_score_minmax
results['f1_score_custom'] = f1_score_custom
results['f1_score_fournier'] = f1_score_fournier
results